In [2]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import dask.dataframe as dd
import json

## Reading the 20 GB File locally using Dask

In [6]:
# Reading the data and specifying the dtype of some columns because Dask can confuse them
df = dd.read_csv(
    'all_data.csv',
    low_memory=False,
    dtype={'latitud': 'object',
   'longitud': 'object',
   'precio': 'object'})

# Removing the incorrect values under estado column as this would be needed to answer Question 4 :)
df = df[(df.estado != 'estado')]
all_data = df.compute()
print(all_data.head(5))

# Creating a pivot to answer question 4
pivot = df.groupby('estado').count()
pivot = pivot.compute()



                  producto                                   presentacion  \
0  CUADERNO FORMA ITALIANA          96 HOJAS PASTA DURA. CUADRICULA CHICA   
1                 CRAYONES              CAJA 12 CERAS. JUMBO. C.B. 201423   
2                 CRAYONES      CAJA 12 CERAS. TAMANO REGULAR C.B. 201034   
3        COLORES DE MADERA  CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646   
4              COLOR LARGO        CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036   

      marca         categoria          catalogo precio  \
0  ESTRELLA  MATERIAL ESCOLAR  UTILES ESCOLARES   25.9   
1   CRAYOLA  MATERIAL ESCOLAR  UTILES ESCOLARES   27.5   
2   CRAYOLA  MATERIAL ESCOLAR  UTILES ESCOLARES   13.9   
3  PINCELIN  MATERIAL ESCOLAR  UTILES ESCOLARES   46.9   
4   CRAYOLA  MATERIAL ESCOLAR  UTILES ESCOLARES    115   

             fechaRegistro     cadenaComercial        giro  \
0  2011-05-18 00:00:00.000  ABASTECEDORA LUMEN  PAPELERIAS   
1  2011-05-18 00:00:00.000  ABASTECEDORA LUMEN  PAPELERIAS   
2 

## Variables used to answer all of the questions

In [ ]:
# Copy the product column to create pivot easily
all_data['sum_product'] = all_data.product

# Variables to answer the questions
store_num = all_data.cadenaComercial.value_counts()[:10]
type_of_store_num = type(store_num)
distinct = all_data.cadenaComercial.nunique()
list_of_states = all_data['estado'].unique()

## Question 1: How many commercial chains are monitored, and therefore, included in this database?

In [10]:
########### ANSWER # 1 ###########
print(f"The number of rows is {all_data.shape[0]}\n")
print(f"Answer 1: The number of stores is monitorized by Profeco is: {distinct}\n")

The number of rows is 62530695

Answer 1: The number of stores is monitorized by Profeco is: 704



## Question 2: What are the top 10 monitored products by State? As the result is quite large, I am saving the output and enclosing it in the repo as an excel file

In [14]:
########### ANSWER # 2 ###########
print(f"Answer 2: What are the top 10 monitored products by State?:\n")
states_frame = []
for state in list_of_states:
    sliced_pivot = all_data[(all_data.estado == state)]
    grouped = sliced_pivot.groupby(['estado', 'producto'])['sum_product'].count().nlargest(10)
    states_frame.append(grouped)
top_products_by_state = pd.concat(states_frame)
top_products_by_state.to_excel("output.xlsx", index=True, header=True)
print("Please check output.xlsx in the repository")

Answer 2: What are the top 10 monitored products by State?:

Please check output.xlsx in the repository


## Question 3: Which is the commercial chain with the highest number of monitored products?

In [15]:
########### ANSWER # 3 ###########
print(f"\nAnswer 3: The store with the highest number of monitored products: {store_num.index[0]}\n")


Answer 3: The store with the highest number of monitored products: WAL-MART



## Question 4: Use the data to find an interesting fact.

### Goal: Create a Choropleth map using folium and geopandas to plot the distribution of the total products tracked in each state

In [23]:
# Renaming some headers so I can join the geospatial Dataframe and Profeco Products dataset
rename_dict = {
    'Coahuila de zaragoza': 'Coahuila',
    'Distrito federal': 'Distrito Federal',
    'Michoacán de ocampo' : 'Michoacán',
    'Nuevo león': 'Nuevo León',
    'Quintana roo': 'Quintana Roo',
    'Veracruz de ignacio de la llave' : 'Veracruz',
    'Baja california' : 'Baja California',
    'Baja california sur' : 'Baja California Sur',
    'San luis potosí' : 'San Luis Potosí'
    
}

# Using the pivot to clean the standardize the data
pivot = pivot[["producto"]]
pivot.index = pivot.index.str.capitalize()
pivot = pivot.rename(index=rename_dict)
pivot.sort_index()
pivot['producto'] = pivot.producto/1000
print(pivot)

                      producto
estado                        
Aguascalientes         628.576
Coahuila              1512.473
Colima                1159.974
Distrito Federal     11284.102
Guanajuato            2638.456
Hidalgo               1017.667
Jalisco               4552.128
Michoacán             2093.037
México                8173.302
Nuevo León            3171.091
Oaxaca                1075.420
Puebla                2021.476
Querétaro             1667.824
Quintana Roo          2076.525
Sinaloa               1720.736
Sonora                1698.620
Tabasco               1842.633
Tlaxcala              2081.024
Veracruz               690.420
Yucatán               2300.994
Zacatecas             1383.201
Durango                563.269
Baja California       1200.999
Baja California Sur    977.128
Campeche               576.079
Chiapas                527.160
Chihuahua              919.673
Guerrero               485.470
Morelos                353.225
Nayarit                419.547
San Luis

In [17]:
## Read the SHP files and creating a Dataframe from it
states = gpd.read_file('Mexico_States/Mexico_States.shp')
states = states.sort_values(by=['NAME'])
states = states.merge(pivot, how='left', left_on=['NAME'], right_on='estado')
states.head(5)

,CODE,NAME,geometry,producto
0,MX01,Aguascalientes,"POLYGON ((-101.84620 22.01176, -101.96530 21.8...",628.576
1,MX02,Baja California,"MULTIPOLYGON (((-113.13972 29.01778, -113.2405...",1200.999
2,MX03,Baja California Sur,"MULTIPOLYGON (((-111.20612 25.80278, -111.2302...",977.128
3,MX04,Campeche,"MULTIPOLYGON (((-91.83446 18.63805, -91.84195 ...",576.079
4,MX05,Chiapas,"POLYGON ((-91.43750 17.24111, -91.35278 17.176...",527.160


In [24]:
# Plotting the raw map of Mexico
my_map = folium.Map(location=[23.634501,-102.552784], zoom_start=5)

# Creating the Choropleth object and adding it to the map
folium.Choropleth(
    geo_data=states,
    name='choropleth',
    data=states,
    columns=['NAME', 'producto'],
    key_on='feature.properties.NAME',
    fill_color="YlGnBu",
    smooth_factor=0,
    Highlight= True,
    show=False,
    overlay=True,
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Tracked products by State'
).add_to(my_map)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# Hover Tag
NIL = folium.features.GeoJson(
    data = states,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME','producto'],
        aliases=['Estado','Productos en Miles'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

# Adding the hover tag
my_map.add_child(NIL)
my_map.keep_in_front(NIL)

my_map

### We can conclude that, as expected, Distrito Federal, Nuevo León and Jalisco are the states with the most weight in the tracked products, however, Yucatán and Guanajuato are also much higher than the rest of the states

## Question 5 what are the lessons learned from this exercise?

### The consumption of large datasets could be difficult to handle, you need to research in the documentation how to workaround issues with memory management, thankfully I could use Dask and memory options to make it work

## 6. Can you identify other ways to approach this problem? Explain.

### In my first submission I used Pandas and chunksize to handle the dataset, but it was a lot of pressure for my PC, so researching online I found Dask as a good workaround because is pretty much a beefy version of Dask, with most of its methods and properties like Pandas.